In [1]:
%matplotlib inline

In [2]:
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import sklearn
import sklearn.linear_model
from datetime import datetime, date
from sklearn_pandas import DataFrameMapper
import warnings
import timeit
from collections import defaultdict, OrderedDict
import tabulate
import time
import GPy
from IPython.display import display
import scipy
from scipy import stats

timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

matplotlib.style.use('ggplot')

RANDOM_SEED = 33

In [29]:
def conditional_coef_matrix(covariance):
    inv_cov = np.linalg.inv(covariance)
    C = np.linalg.inv(np.diag(np.diag(inv_cov))).dot(inv_cov) 
    return np.eye(covariance.shape[0]) - C, inv_cov


def conditional_distributions(mean, covariance):
    C, inv_cov = conditional_coef_matrix(covariance)
    results = []
    
    for i, mu_i in enumerate(mean):
        components = []
        components.append(r'x_%d | \{x_j\}_{j \neq %d}' % (i + 1, i + 1))
        components.append(r'\sim \mathcal{N}(')
        components.append('{mu}'.format(mu=mu_i))
        for j, mu_j in enumerate(mean):
            c_ij = C[i, j]
            if i == j or 0 == c_ij:
                continue
            
            components.append('+')
            
            if 1 != c_ij:
                components.append('{c_ij:.3f}('.format(c_ij=c_ij))    
                
            components.append(r'x_{%d} - %d' % (j + 1, mu_j))
            
            if 1 != c_ij:
                components.append(')')
        
        components.append(r',\ {stddev:.3f}'.format(stddev=1.0 / (inv_cov[i, i])))
        components.append(')')
        
        results.append(' '.join(components))
        
    [print(x) for x in results]

In [30]:
conditional_distributions(np.asarray([1, 1]), np.asarray([[2, 1], [1, 2]]))

x_1 | \{x_j\}_{j \neq 1} \sim \mathcal{N}( 1 + 0.500( x_{2} - 1 ) ,\ 1.500 )
x_2 | \{x_j\}_{j \neq 2} \sim \mathcal{N}( 1 + 0.500( x_{1} - 1 ) ,\ 1.500 )


$$ x_1 | \{ x_j \}_{j \neq 1} \sim \mathcal{N}( 1 + 0.5( x_{2} - 1 ) ,\ 1.500 )  $$
$$ x_2 | \{x_j\}_{j \neq 2} \sim \mathcal{N}( 1 + 0.5( x_{1} - 1 ) ,\ 1.500 ) $$



In [31]:
conditional_distributions(np.asarray([1, 1]), np.asarray([[3, 0], [0, 0.5]]))

x_1 | \{x_j\}_{j \neq 1} \sim \mathcal{N}( 1 ,\ 3.000 )
x_2 | \{x_j\}_{j \neq 2} \sim \mathcal{N}( 1 ,\ 0.500 )


$$ x_1 | \{x_j\}_{j \neq 1} \sim \mathcal{N}( 1 ,\ 3.000 ) $$
$$ x_2 | \{x_j\}_{j \neq 2} \sim \mathcal{N}( 1 ,\ 0.500 ) $$

In [32]:
conditional_distributions(np.asarray([1, 1]), np.asarray([[2, 1.9], [1.9, 2]]))

x_1 | \{x_j\}_{j \neq 1} \sim \mathcal{N}( 1 + 0.950( x_{2} - 1 ) ,\ 0.195 )
x_2 | \{x_j\}_{j \neq 2} \sim \mathcal{N}( 1 + 0.950( x_{1} - 1 ) ,\ 0.195 )


$$ x_1 | \{x_j\}_{j \neq 1} \sim \mathcal{N}( 1 + 0.950( x_{2} - 1 ) ,\ 0.195 ) $$
$$ x_2 | \{x_j\}_{j \neq 2} \sim \mathcal{N}( 1 + 0.950( x_{1} - 1 ) ,\ 0.195 ) $$